In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/interested_no_profile.csv')
# df = pd.read_csv('data/interested_no_profile_whole_behavior.csv')

In [3]:
df.columns

Index([u'distinct_id', u'event', u'browser', u'ip', u'latest_utm_medium',
       u'latest_utm_source', u'os', u'ref_host_pageview', u'screen_height',
       u'screen_width', u'current_page', u'session_referrer_host',
       u'url_to_submit', u'isSuccess', u'name', u'btnClick_page',
       u'pagePosition', u'pageStayTime', u'project_name', u'requestBtn',
       u'result', u'session_page_url', u'site_url', u'time', u'Bj_timestamp',
       u'dayofweek', u'hourofday', u'date', u'new_user'],
      dtype='object')

In [4]:
distinct_id = df.distinct_id.unique()
pattern = df.groupby(by = 'distinct_id').apply(lambda x: x.sort_values('Bj_timestamp'))

In [5]:
# pattern[(pattern.distinct_id == '9939d3e087bca29c42334d96dccd25ca0e06652a')][['btnClick_page', 'name', 'event', 'current_page','Bj_timestamp']]

In [6]:
req_name_list = ['request', 'cli_document', 'cli_video', 'cli_blog','last_name', 
                 'cli_demo', 'cli_logo', 'cli_product', 'cli_about','cli_bround', 'cli_mail']

In [7]:
event_list = ['$pageview', 'btnClick', 'click_send_cellphone','verify_cellphone_code', 'index_leave', 'clickSubmit', 'demo_leave',
       'about_leave', 'courses_leave', 'formSubmit', 'page_close','courses_play_leave']

In [8]:
cur_page_list = ['homepage', 'demo', 'lookmanual', 'index', 'product','about', 'bround', 'manual', 'tool', 
                 'courses', 'tools_page_close','play_video', 'share']

In [9]:
new_fea = {'dis_id': list(), 
           'new_user': list(),
           'browser': list(), 'ip': list(), 'os': list(),
           
           # about button click
           'behavior_list': list(),           
           'request': list(),'cli_demo': list(),'cli_product': list(),
           'cli_document': list(),'cli_bround': list(),'last_name': list(),
           'cli_about': list(),'cli_blog': list(),'cli_video': list(),'cli_logo': list(),'cli_mail': list(),
           
           # about event
           'event_order': list(),
           '$pageview': list(), 'btnClick': list(), 'click_send_cellphone': list(),
           'verify_cellphone_code': list(), 'index_leave': list(), 'clickSubmit': list(), 'demo_leave': list(),
           'about_leave': list(), 'courses_leave': list(), 'formSubmit': list(), 'page_close': list(),
           'courses_play_leave': list(),
           
           # about current page
           'page_order': list(),'tot_page_time': list(),
           'homepage': list(), 'demo': list(), 'lookmanual': list(), 'index': list(), 'product': list(),
           'about': list(), 'bround': list(), 'manual': list(), 'tool': list(), 'courses': list(), 'tools_page_close': list(),
           'play_video': list(), 'share': list(),
           
           # project
           'project': list(), 'num_project': list(),
           
           # utm
           'latest_utm_source': list(),
           'latest_utm_medium': list(),
           
           'session_referrer_host': list(),
           'ref_host_pageview': list(),
           'url_to_submit': list(),
           
           'Mo': list(),'Tu': list(), 'We': list(),'Th': list(),
            'Fr': list(),'Sa': list(),'Su': list(),
           
           'wrong_num': list(),
           'less_1min': list(),
           'send_suc': list(),
           
           'wrong_code': list(),
           'no_num': list(),
           'ver_suc': list(),
           
           'num_fail_submit': list(),
           'isSuccess': list()
          }

for _id in distinct_id:
    print _id
    demo_user = pattern[(pattern.distinct_id == _id)]
    
    new_fea['dis_id'].append(demo_user.distinct_id.values[0])
    new_fea['browser'].append(demo_user.browser.values[0])
    new_fea['ip'].append(demo_user.ip.value_counts().idxmax())
    new_fea['os'].append(demo_user.os.value_counts().idxmax())
    
    # about button click
    req_name = demo_user[(demo_user.name != 'None')].name
#     new_fea['btn_order'].append(req_name.values)
    name_count = req_name.value_counts()
    for name in req_name_list:
        new_fea[name].append(0)
    for name in name_count.index:
        if name in req_name_list:
            new_fea[name][-1] = name_count[name]
        
    # behavior list
#     beh_df = demo_user[(demo_user.event == 'btnClick')]
    behave = list()
#     for _, row in beh_df.iterrows():
    for _, row in demo_user.iterrows():
        if row.btnClick_page != 'None':
            behave.append(row.btnClick_page)
        if row['name'] != 'None':
            behave.append(row['name'])
    new_fea['behavior_list'].append(behave)
    
    # about event 
    event = demo_user.event
    new_fea['event_order'].append(event.values)
    event_count = event.value_counts()
    for ev in event_list:
        new_fea[ev].append(0)
    for ev in event_count.index:
        new_fea[ev][-1] = event_count[ev]

    # about current page
    cur_page = demo_user.loc[(demo_user['current_page'] != 'None')].current_page
    new_fea['page_order'].append(cur_page.values)
    page_count = cur_page.value_counts()
    for page in cur_page_list:
#         print page
#         print type(new_fea[page])
        new_fea[page].append(0)
    for page in page_count.index:
        new_fea[page][-1] = page_count[page]

    
    latest_utm_source = demo_user.loc[(demo_user['latest_utm_source'] != 'None'), 'latest_utm_source'].value_counts()
    if latest_utm_source.shape[0] == 0:
        new_fea['latest_utm_source'].append(None)
    else:
        new_fea['latest_utm_source'].append(latest_utm_source.idxmax())

    latest_utm_medium = demo_user.loc[(demo_user['latest_utm_medium'] != 'None'), 'latest_utm_medium'].value_counts()
    if latest_utm_medium.shape[0] == 0:
        new_fea['latest_utm_medium'].append(None)
    else:
        new_fea['latest_utm_medium'].append(latest_utm_medium.idxmax())

    session_referrer_host = demo_user.loc[(demo_user['session_referrer_host'] != 'None'), 'session_referrer_host'].value_counts()
    if session_referrer_host.shape[0] == 0:
        new_fea['session_referrer_host'].append(None)
    else:
        new_fea['session_referrer_host'].append(session_referrer_host.idxmax())

    ref_host_pageview = demo_user.loc[(demo_user['ref_host_pageview'] != 'None'), 'ref_host_pageview'].value_counts()
    if ref_host_pageview.shape[0] == 0:
        new_fea['ref_host_pageview'].append(None)
    else:
        new_fea['ref_host_pageview'].append(ref_host_pageview.idxmax())

    url_to_submit = demo_user['url_to_submit'].value_counts()
    if url_to_submit.shape[0] == 0:
        new_fea['url_to_submit'].append(None)
    else:
        new_fea['url_to_submit'].append(url_to_submit.idxmin())

    isSuc = demo_user.loc[(demo_user.isSuccess.notnull()), 'isSuccess'].values
    suc = 1 if True in isSuc else 0
    new_fea['isSuccess'].append(suc)
    new_fea['num_fail_submit'].append(len(isSuc) - suc)

    new_fea['tot_page_time'].append(demo_user.loc[(demo_user.pageStayTime.notnull())].pageStayTime.values.sum())
    
    pro = demo_user.project_name.value_counts()
    if pro.shape[0] > 0:
        new_fea['project'].append(pro.index)
        new_fea['num_project'].append(pro.shape[0])
    else:
        new_fea['project'].append('None')
        new_fea['num_project'].append(0)

    res_count = demo_user['result'].value_counts()
    if 'ajax success' in res_count.index:
        new_fea['send_suc'].append(1)
    else:
        new_fea['send_suc'].append(0)

    if '手机号格式错误' in res_count.index:
        new_fea['wrong_num'].append(res_count['手机号格式错误'])
    else:
        new_fea['wrong_num'].append(0)

    if '距上次发送时间不到 1 分钟' in res_count.index:
        new_fea['less_1min'].append(res_count['距上次发送时间不到 1 分钟'])
    else:
        new_fea['less_1min'].append(0)

    if '正确' in res_count.index:
        new_fea['ver_suc'].append(1)
    else:
        new_fea['ver_suc'].append(0)

    if '验证码错误' in res_count.index:
        new_fea['wrong_code'].append(res_count['验证码错误'])
    else:
        new_fea['wrong_code'].append(0)

    if '没有该号码验证信息' in res_count.index:
        new_fea['no_num'].append(res_count['没有该号码验证信息'])
    else:
        new_fea['no_num'].append(0)

    days = demo_user.dayofweek.unique()
    if 1 in days:
        new_fea['Mo'].append(1)
    else:
        new_fea['Mo'].append(0)

    if 2 in days:
        new_fea['Tu'].append(1)
    else:
        new_fea['Tu'].append(0)

    if 3 in days:
        new_fea['We'].append(1)
    else:
        new_fea['We'].append(0)

    if 4 in days:
        new_fea['Th'].append(1)
    else:
        new_fea['Th'].append(0)

    if 5 in days:
        new_fea['Fr'].append(1)
    else:
        new_fea['Fr'].append(0)

    if 6 in days:
        new_fea['Sa'].append(1)
    else:
        new_fea['Sa'].append(0)

    if 7 in days:
        new_fea['Su'].append(1)
    else:
        new_fea['Su'].append(0)

    if demo_user.new_user.values[0]:
        new_fea['new_user'].append(1)
    else:
        new_fea['new_user'].append(0)

9939d3e087bca29c42334d96dccd25ca0e06652a
ac2b15e51b9d22b6b46782fbde206657db0c1a26
91f49ce17152389fc9d94df066f595ff892e8a88
3b91e81c8b9783f10faa1d0c8b3422f4ab5f4c9e
9357ebad253cd4e8343ae218552381d97144f81d
b8a7a2cfe58e97364085377c82a31e5b7fe00df6
6d06085a4fb2abfc8ddd8c9ff7315e74d8cd6d69
130dcea7ba6d3e75e6d55e62df8c7879df678214
f3b8662af74b4aea2f5ef5e2d059fd046b726891
a630476c5f5269399a171498a7daf32c56a76bfe
658ade996a1b32bc98cfc15782129df02b3ff161
db4776f5cb1133a811a93d2892fc75644a4c7e95
cc3d3ee6a596a4e7535096707baf1bd908901fb2
fb34f0eab722bf9efd12ee813a32e004ae8d0277
360620398ea512c84070b1ab5a7005f05acb398d
fcb6652f34efe41b93d3ac5136ae00f4b97dade0
a6c7952ddb38ffb2f5a2262bc3d397cf76b08731
1ab6424b5649a0b4865fb960798a8a2c12c504e5
75aa793601cd63aac7666028a2868797df1f501d
d5d8e28c5452ea35b61a491e49e5556c20f33233
05275c9a86246309de06ef9e9443dbecb2d9e432
2c321b62fa23788b83fb8ba5186e770d1f98da63
0d76d77a856a471d4d44b5ce9e1985f1e7b96582
8a9c2c8bb6127a747c58e574563c32669af437f4
edeea136c6ccb483

In [10]:
cols = ['dis_id','new_user','browser', 'ip', 'os',
           
           # about button click
           'behavior_list',         
           'request','cli_demo','cli_product',
           'cli_document','cli_bround','last_name',
           'cli_about','cli_blog','cli_video','cli_logo','cli_mail',
           
           # about event
           'event_order',
           '$pageview', 'btnClick', 'click_send_cellphone',
           'verify_cellphone_code', 'index_leave', 'clickSubmit', 'demo_leave',
           'about_leave', 'courses_leave', 'formSubmit', 'page_close',
           'courses_play_leave',
           
           # about current page
           'page_order','tot_page_time',
           'homepage', 'demo', 'lookmanual', 'index', 'product',
           'about', 'bround', 'manual', 'tool', 'courses', 'tools_page_close',
           'play_video', 'share',
           
           # project
           'project', 'num_project',
           
           # utm
           'latest_utm_source',
           'latest_utm_medium',
           
           'session_referrer_host',
           'ref_host_pageview',
           'url_to_submit',
           
           'Mo','Tu', 'We','Th',
            'Fr','Sa','Su',
           
           'wrong_num',
           'less_1min',
           'send_suc',
           
           'wrong_code',
           'no_num',
           'ver_suc',
           
           'num_fail_submit',
           'isSuccess'
        ]

In [11]:
user_df = pd.DataFrame(data = new_fea, columns = cols)

In [12]:
user_df.head(4)

,dis_id,new_user,browser,ip,os,behavior_list,request,cli_demo,cli_product,cli_document,...,Sa,Su,wrong_num,less_1min,send_suc,wrong_code,no_num,ver_suc,num_fail_submit,isSuccess
0,9939d3e087bca29c42334d96dccd25ca0e06652a,1,chrome,111.204.198.242,windows,"[index, request, index, request, index, reques...",3,2,1,1,...,0,0,0,0,1,0,0,1,0,0
1,ac2b15e51b9d22b6b46782fbde206657db0c1a26,1,chrome,58.62.134.182,windows,"[index, cli_blog]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,91f49ce17152389fc9d94df066f595ff892e8a88,1,chrome,121.8.201.114,macosx,"[demo, cli_demo, demo, cli_demo, demo, cli_dem...",1,6,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3b91e81c8b9783f10faa1d0c8b3422f4ab5f4c9e,1,others,223.24.43.9,Android,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
user_df.behavior_list[0]

['index',
 'request',
 'index',
 'request',
 'index',
 'request',
 'last_name',
 'index',
 'cli_demo',
 'demo',
 'cli_product',
 'cli_document',
 'cli_demo']

In [14]:
user_df.shape

(4868, 67)

In [15]:
user_df.isSuccess.value_counts()

0    4556
1     312
Name: isSuccess, dtype: int64

In [16]:
user_df.to_csv('data/users_data.csv', index = False, encoding = 'utf-8')
# user_df.to_csv('data/users_data_whole_behavior.csv', index = False, encoding = 'utf-8')

    improvement:
    different page with different importance, not equally
    time zoom: morning, afternoon, evening (0/1)